In [1]:
import tuxml
import pandas as pd 

df = tuxml.load_dataset()
df.replace(2, 0, inplace=True)

In [2]:
df.sort_values("vmlinux")[:5]

,OPENVSWITCH,TEXTSEARCH_FSM,NETFILTER_XT_MATCH_TCPMSS,MPLS,NFC_HCI,NETFILTER_XT_MATCH_TIME,NET_MPLS_GSO,NFC_SHDLC,NETFILTER_XT_MATCH_U32,OPENVSWITCH_GRE,...,LZO-vmlinux,LZO,LZ4-bzImage,LZ4-vmlinux,LZ4,cid,nbyes,nbno,nbmodule,nbyesmodule
109097,0,0,0,0,0,0,0,0,0,0,...,2863640,644621,814544.0,2900824,669288,124978,25,9210,0,25
58621,0,0,0,0,0,0,0,0,0,0,...,2777624,558712,724432.0,2810712,578375,74458,24,9211,0,24
107213,0,0,0,0,0,0,0,0,0,0,...,3953176,1734208,1986512.0,4072280,1841339,123091,525,8041,669,1194
99232,0,0,0,0,0,0,0,0,0,0,...,4248440,2027905,2290640.0,4375728,2144007,115110,420,8104,711,1131
99777,0,0,0,0,0,0,0,0,0,0,...,4096536,1877790,2139088.0,4223832,1994471,115655,505,8324,406,911


In [3]:
df_importance = pd.read_csv("feature_importanceRF.csv", header=None)
df_importance.rename(columns={0 : 'option', 1 : 'importance' }, inplace=True)
# df_importance = df[df_importance[:100]["Unnamed: 0"].values]
df_importance = df[df_importance["option"].values] # [:100]
# df_importance.head()
df_importance

,DEBUG_INFO,nbyes,DEBUG_INFO_REDUCED,DEBUG_INFO_SPLIT,X86_NEED_RELOCS,RANDOMIZE_BASE,UBSAN_SANITIZE_ALL,KASAN_OUTLINE,KASAN,UBSAN_ALIGNMENT,...,RSI_DEBUGFS,CRYPTO_RNG_DEFAULT,CRYPTO_ECB,CRYPTO_WORKQUEUE,HISAX_HFC_SX,USB_GSPCA_VC032X,HISAX_ENTERNOW_PCI,CRYPTO_HASH2,HISAX_DEBUG,IP_NF_ARP_MANGLE
14758,0,1219,0,0,1,1,0,1,1,0,...,0,1,1,1,0,0,0,1,0,0
14759,0,1166,0,0,0,0,0,0,0,1,...,0,1,1,1,0,0,0,1,0,0
14760,0,1410,0,0,0,0,0,0,0,0,...,0,1,1,1,0,0,0,1,0,0
14761,0,1924,0,0,0,0,0,0,0,1,...,0,1,1,1,0,0,0,1,0,0
14762,0,2435,0,0,0,0,0,0,0,0,...,0,1,1,1,0,0,0,1,0,0
14763,0,1375,0,0,0,0,0,0,0,0,...,0,1,1,1,1,0,0,1,1,0
14764,0,1899,0,0,0,0,0,0,0,0,...,0,1,1,1,0,0,0,1,0,0
14765,0,3085,0,0,0,0,0,1,1,1,...,0,1,1,1,0,0,0,1,0,0
14766,1,1294,0,0,0,0,0,0,0,0,...,0,1,1,1,0,0,0,1,0,0
14767,0,2291,0,0,0,0,0,0,0,0,...,0,1,1,1,0,0,0,1,0,0


In [4]:
size_methods = ["vmlinux", "GZIP-bzImage", "GZIP-vmlinux", "GZIP", "BZIP2-bzImage", 
              "BZIP2-vmlinux", "BZIP2", "LZMA-bzImage", "LZMA-vmlinux", "LZMA", "XZ-bzImage", "XZ-vmlinux", "XZ", 
              "LZO-bzImage", "LZO-vmlinux", "LZO", "LZ4-bzImage", "LZ4-vmlinux", "LZ4"]
additional_ft = ["nbyes", "nbno", "nbmodule", "nbyesmodule"]

In [5]:
#learning a model by using linear regression
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.linear_model import ElasticNet

def run_regressorML(reg, test_size, size_target, x_train, x_test):
    assert(size_target in size_methods)
    reg.fit(x_train, y_train)
    y_pred = reg.predict(x_test)
    #y_pred = reg.intercept_ + np.sum(reg.coef_ * x_test.values, axis=1)
    dfErrors = pd.DataFrame({"error":(y_pred - y_test).abs(), "% error":((y_pred - y_test)/y_test).abs()*100})
    return dfErrors["% error"].describe()

In [6]:
def ft_importances(coef_, col):
    importanceSeries = pd.Series(coef_, index=col.values)
    return importanceSeries[importanceSeries != 0].abs().sort_values(ascending = False)

In [7]:
df_result  = pd.DataFrame(columns = ['algorithm_name', 'ft_dummyfication', 'origin_ft_selection', 'ft_selection', 'hyperparameters', 'size_target', 'additional_ft', 'test_size', 'coef_order', 'accuracy'])

In [8]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import ElasticNet
from sklearn import linear_model
from sklearn import metrics

target = "vmlinux"
paramsLasso = {'alpha': [25,10,4,2,1.0,0.8,0.5,0.3,0.2,0.1,0.05,0.02,0.01],
               'max_iter': [1, 5, 10, 100, 1000],}
paramsRidge = {'alpha': [25,10,4,2,1.0,0.8,0.5,0.3,0.2,0.1,0.05,0.02,0.01]}
paramsENet = {'alpha': [25,10,4,2,1.0,0.8,0.5,0.3,0.2,0.1,0.05,0.02,0.01],
              'l1_ratio': [0.0, 0.5, 1.0, 0.1],
              'max_iter': [1, 5, 10, 100, 1000],}

for ft_dummyfication in [False]: # [False, True]:
    if ft_dummyfication:
        #Creating dummy variables in pandas
        df = pd.get_dummies(df, columns=list(df.drop(columns=["cid"]).drop(columns=size_methods).drop(columns=additional_ft).columns.values))
        df_importance = pd.get_dummies(df_importance, columns=list(df_importance.drop(columns=["nbyes", "nbno", "nbyesmodule"]).columns.values))
        
    ft_selection =  {'None': df.drop(columns=["cid"]).drop(columns=size_methods).columns,
                     'RandomForest': df_importance.columns[:100],}
        
    for origin_ft_selection in ft_selection:
        print(origin_ft_selection)
        # for size in [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]:
        for size in [0.99]:
            print(size)

            x_train, x_test, y_train, y_test = train_test_split(df[ft_selection[origin_ft_selection]], df[target], test_size=size, random_state=42)  
            # x_train, y_train = np.array(x_train.values), np.array(y_train.values)

            model = {'LinearRegression': LinearRegression(),
                    # 'Lasso': GridSearchCV(linear_model.Lasso(), param_grid=paramsLasso, cv=3).fit(x_train, y_train).best_estimator_,
                    # 'Ridge': GridSearchCV(linear_model.Ridge(), param_grid=paramsRidge, cv=3).fit(x_train, y_train).best_estimator_,
                    # 'ElasticNet': GridSearchCV(ElasticNet(), param_grid=paramsENet, scoring='r2', cv=3).fit(x_train, y_train).best_estimator_,
                    'PolynomialRegression': LinearRegression()
                    }
        
            for key in model:
                if (key == 'PolynomialRegression') and (origin_ft_selection == 'None'):
                    print("It doesn't scale")
                else:
                    if (key == 'PolynomialRegression'):
                        #for 2 options we have: 𝑓(𝑥₁, 𝑥₂) = 𝑏₀ + 𝑏₁𝑥₁ + 𝑏₂𝑥₂ + 𝑏₃𝑥₁² + 𝑏₄𝑥₁𝑥₂ + 𝑏₅𝑥₂²
                        pf = PolynomialFeatures(degree=2, include_bias=False)
                        x_train_pf = pf.fit_transform(x_train)
                        x_test_pf = pf.fit_transform(x_test)
                        res_model = run_regressorML(model[key], test_size=size, size_target=target, x_train=x_train_pf, x_test=x_test_pf)
                        # model[key].coef_ = model[key].coef_[:100]
                        importanceSeries = pd.Series(model[key].coef_, index=pf.get_feature_names(x_train.columns))
                        coef_order = importanceSeries[importanceSeries != 0].abs().sort_values(ascending = False)
                    else:
                        res_model = run_regressorML(model[key], test_size=size, size_target=target, x_train=x_train, x_test=x_test)
                        coef_order = ft_importances(model[key].coef_, col=ft_selection[origin_ft_selection])
                   
                        
                    df_result.loc[len(df_result)] = [key, ft_dummyfication, origin_ft_selection, ft_selection[origin_ft_selection], model[key], target, additional_ft, size, coef_order, res_model]

df_result.to_csv("results.csv", header=True)

None
0.99
It doesn't scale
RandomForest
0.99


In [9]:
res = pd.read_csv('results.csv')
res

,Unnamed: 0,algorithm_name,ft_dummyfication,origin_ft_selection,ft_selection,hyperparameters,size_target,additional_ft,test_size,coef_order,accuracy
0,0,LinearRegression,False,None,"Index(['OPENVSWITCH', 'TEXTSEARCH_FSM', 'NETFI...","LinearRegression(copy_X=True, fit_intercept=Tr...",vmlinux,"['nbyes', 'nbno', 'nbmodule', 'nbyesmodule']",0.99,DEBUG_INFO 1.354154e+07\...,count 91547.000000\nmean 95.035678\n...
1,1,LinearRegression,False,RandomForest,"Index(['DEBUG_INFO', 'nbyes', 'DEBUG_INFO_REDU...","LinearRegression(copy_X=True, fit_intercept=Tr...",vmlinux,"['nbyes', 'nbno', 'nbmodule', 'nbyesmodule']",0.99,nbno 3.710936e+1...,count 91547.000000\nmean 50.042485\n...
2,2,PolynomialRegression,False,RandomForest,"Index(['DEBUG_INFO', 'nbyes', 'DEBUG_INFO_REDU...","LinearRegression(copy_X=True, fit_intercept=Tr...",vmlinux,"['nbyes', 'nbno', 'nbmodule', 'nbyesmodule']",0.99,DEBUG_INFO_REDUCED DEBUG_INFO_SPLIT ...,count 91547.000000\nmean 54.656842\n...


In [12]:
# oblige to reparse 
coeffs = res.iloc[2]['coef_order']
coef = pd.DataFrame(columns=['option', 'coefficient'])
i = 0
coeffs.splitlines()
for coeff in coeffs.splitlines():
    print(coeff)
    optcoeff = [c for c in coeff.split(' ') if c is not '']
    if len(optcoeff) == 1:
        continue
    elif len(optcoeff) == 3:
        opt = optcoeff[0] + " " + optcoeff[1]
        coeff = optcoeff[2]
    else: # standard: one option, one coeff
        opt = optcoeff[0]
        coeff = optcoeff[1]
    coef.loc[i] = (opt, coeff)
    i = i + 1
coef

DEBUG_INFO_REDUCED DEBUG_INFO_SPLIT           1.642945e+07
DEBUG_INFO X86_NEED_RELOCS                    1.598978e+07
DEBUG_INFO RANDOMIZE_BASE                     1.598978e+07
DEBUG_INFO_SPLIT UBSAN_NULL                   9.746300e+06
DEBUG_INFO DRM_I915_USERPTR                   9.629731e+06
UBSAN_SANITIZE_ALL UBSAN_ALIGNMENT            9.613293e+06
DEBUG_INFO FB_IMSTT                           8.400341e+06
DEBUG_INFO MMU_NOTIFIER                       8.276061e+06
X86_NEED_RELOCS DRM_I915_USERPTR              8.029887e+06
RANDOMIZE_BASE DRM_I915_USERPTR               8.029887e+06
DEBUG_INFO DRM_VM                             7.625486e+06
DEBUG_INFO DRM_NOUVEAU                        7.540458e+06
GCOV_PROFILE_ALL KCOV_INSTRUMENT_ALL          7.521695e+06
DEBUG_INFO GPIO_SCH311X                       7.395497e+06
RANDOMIZE_BASE DRM_HISI_HIBMC                 7.262517e+06
X86_NEED_RELOCS DRM_HISI_HIBMC                7.262517e+06
RANDOMIZE_BASE DRM_RADEON_USERPTR             7.147965e+

,option,coefficient
0,DEBUG_INFO_REDUCED DEBUG_INFO_SPLIT,1.642945e+07
1,DEBUG_INFO X86_NEED_RELOCS,1.598978e+07
2,DEBUG_INFO RANDOMIZE_BASE,1.598978e+07
3,DEBUG_INFO_SPLIT UBSAN_NULL,9.746300e+06
4,DEBUG_INFO DRM_I915_USERPTR,9.629731e+06
5,UBSAN_SANITIZE_ALL UBSAN_ALIGNMENT,9.613293e+06
6,DEBUG_INFO FB_IMSTT,8.400341e+06
7,DEBUG_INFO MMU_NOTIFIER,8.276061e+06
8,X86_NEED_RELOCS DRM_I915_USERPTR,8.029887e+06
9,RANDOMIZE_BASE DRM_I915_USERPTR,8.029887e+06


In [ ]:
# Boxplot
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

data = [order_lr, order_lasso, order_ridge, order_en]
fig1, ax1 = plt.subplots()
ax1.set_title('Coeficient of feature importance for all algorithms')
ax1.boxplot(data)
plt.show()